# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install foolbox
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# import cleverhans as ch
import foolbox as fb
import cv2
import foolbox.attacks as fatt
# from art.attacks.evasion import FastGradientMethod
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG19
import eagerpy as ep
from PIL import Image, ImageOps

     |████████████████████████████████| 1.7MB 5.4MB/s 
     |████████████████████████████████| 163kB 15.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


# Test Adversarial Images on Baseline VGG19

In [ ]:
# Load images from directory into validation set

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#  "/content/drive/MyDrive/DeepLearning/DeepLearningFinal/validation",
  "/content/drive/MyDrive/DeepLearningFinal/validation",
  validation_split=0.2,
  subset="validation",
  seed = 123,
  shuffle=True,
  image_size=(224, 224),
  batch_size=1)

class_names = val_ds.class_names
# print(class_names)

Found 385 files belonging to 2 classes.
Using 77 files for validation.


In [ ]:
# Load baseline VGG19 model and convert to foolbox model

vgg = tf.keras.applications.VGG19(weights='imagenet')

preprocessing = dict()
bounds = (0,255)
##NEEDS GPU ACTIVATED

fmodel = fb.TensorFlowModel(vgg, bounds=bounds, preprocessing=preprocessing)

574717952/574710816 [==============================] - 35s 0us/step
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [ ]:
# extract images and labels from validation set and convert human-readable labels
# to ImageNet class IDs
images, labels = [], []
class_ids = {'bikes': 671, 'ships': 510}
for img, label in val_ds:
  class_id = class_ids[class_names[int(label)]]
  if class_id == 671:
    labels.append(class_id)
    images.append(img)

In [ ]:
# convert images and labels to tensors

images = np.stack(images)
images = images.reshape(len(images),224,224,3)
images = tf.convert_to_tensor(images)
labels = np.stack(labels).reshape(len(images))
labels = tf.convert_to_tensor(labels)
# fb.utils.accuracy(fmodel, images, labels)

In [ ]:
images = tf.keras.applications.vgg19.preprocess_input(images)

In [ ]:
# run model on adversarial images
# attack = fatt.FGSM()
# epsilons = [
#         0.0,
#         0.1,
#         0.3,
#         0.5,
#         1.0,
#     ]
# raw, clipped, is_adv = attack(fmodel, images, labels,epsilons=epsilons)
# robust_accuracy = 1 - np.mean(is_adv,axis=-1)
# print("robust accuracy for perturbations with")
# for eps, acc in zip(epsilons, robust_accuracy):
#     print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

ValueError: ignored

In [ ]:

# for eps,image_arr,label in zip(epsilons, raw,labels):
#   img_cnt = 1
#   for image in image_arr:
#     image = image.numpy()
#     # plt.imshow(image)
#     save_path = "/content/drive/MyDrive/DeepLearningFinal/adversarial_images/deepfoolL2/ships_adv_eps_" +str(eps)+"/"+str(img_cnt)+".jpg"
#     # print(save_path)
#     plt.imsave(save_path,image/255) 
#     img_cnt +=1


FileNotFoundError: ignored

In [ ]:
# run model on adversarial images
attack = fatt.L2CarliniWagnerAttack()
adv_image = attack.run(fmodel, images, labels)

In [ ]:
img_cnt = 1
for image in adv_image:
    image = image.numpy()
    # plt.imshow(image)
    save_path = "/content/drive/MyDrive/DeepLearningFinal/adversarial_images/CarliniWagner/bikes/"+str(img_cnt)+".jpg"
    # print(save_path)
    plt.imsave(save_path,image/255) 
    img_cnt +=1